In [ ]:
!pip install -q transformers
!pip install -q datasets transformers huggingface_hub
!pip install -q top2vec
!pip install -q top2vec[sentence_encoders]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 20.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 100.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import os
import json

from tqdm import tqdm

In [ ]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH_PROJECT = '''/content/drive/MyDrive/dsc_261'''
PATH_DATA = os.path.join(PATH_PROJECT, 'datasets')

PATH_TWEET_DF = os.path.join(PATH_DATA, 'tweet_dfs')

In [ ]:
aggregated_folder = 'AGGREGATED_DATA'
PATH_AGG_DATA = os.path.join(PATH_DATA, aggregated_folder)
if not os.path.exists(PATH_AGG_DATA):
    os.mkdir(PATH_AGG_DATA)

In [ ]:
raw_files = [x for x in os.listdir(PATH_TWEET_DF) if 'raw' in x and 'csv' in x and x.count('_') == 1]
prep_files = [x for x in os.listdir(PATH_TWEET_DF) if 'prep' in x and 'csv' in x and x.count('_') == 1]

In [ ]:
def get_tickers_from_text(text):
    if (type(text) != str) or (len(text)<=1):
        return []
    try:
        potential_tickers = list(set([word[1:] for word in text.split() if '$' == word[0]]))
        #potential_tickers = [w for w in potential_tickers if w[0].isalpha() ]
        return potential_tickers
    except:
        return []

def cleanPipeTweet(text):
    if (type(text) != str) or (len(text)<=1):
        return ''

    try:

        text = ' '.join(
            [
                word.lower()
                for word in text.split()
                if ((word[0] not in ['@','#','~'])
                    and (word[:2] not in ['--'])
                    and ('http' not in word))
            ]
        )
        text = ' '.join([word[1:] if word[0] == '$' else word for word in text.split()])
        text = ''.join([l for l in list(text) if l not in [',','.']])
        return text

    except:
        return ''


In [ ]:
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# CLEANING TEXT FOR ALL FILES
all_data = pd.DataFrame()
for file_name in tqdm(raw_files):
    file_data = pd.read_csv(os.path.join(PATH_TWEET_DF, file_name), lineterminator='\n')
    file_data['stocks_mentioned'] = file_data['text'].apply(lambda text: get_tickers_from_text(text))
    file_data['text_clean'] = file_data['text'].apply(lambda text: cleanPipeTweet(text))
    all_data = all_data.append(file_data)
    file_data.to_csv(file_name, index=False)

all_data = all_data.reset_index(drop=True)

100%|██████████| 87/87 [00:25<00:00,  3.36it/s]


## GENERATE SENTIMENT LABEL AND SENTIMENT CONFIDENCE SCORE FOR DESIRED **text_clean**:

In [ ]:
%%time
import sys
PATH_UTILITIES = os.path.join(PATH_PROJECT, 'notebooks','src')
sys.path.append(PATH_UTILITIES)

%run $PATH_UTILITIES/'utilities.py'

model="distilbert-base-uncased-finetuned-sst-2-english"
generate_sentiment_label_score(model, all_data,'text_clean')

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

CPU times: user 3h 16min 33s, sys: 24.5 s, total: 3h 16min 58s
Wall time: 49min 38s


,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,date_file,retweeted_status,stocks_mentioned,text_clean,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status,text_clean sentiment_label,text_clean sentiment_score
0,Tue Mar 01 15:58:13 +0000 2016,704697258223095808,704697258223095808,clayton: AGFS 4.89 Agrofresh Solutions Inc $AG...,"<a href=""http://dlvr.it"" rel=""nofollow"">dlvr.i...",False,NaN,NaN,NaN,NaN,...,2016-03-01,NaN,[AGFS],clayton: agfs 489 agrofresh solutions inc agfs...,NaN,NaN,NaN,NaN,NEGATIVE,0.982623
1,Wed Nov 11 19:14:27 +0000 2015,664521588868562947,664521588868562947,$AGFS steady accumulation,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,NaN,NaN,NaN,NaN,...,2015-11-11,NaN,[AGFS],agfs steady accumulation,NaN,NaN,NaN,NaN,NEGATIVE,0.698450
2,Wed Mar 23 03:48:58 +0000 2016,712486269494886400,712486269494886400,AgroFresh Solutions Inc $AGFS Director Buys $2...,"<a href=""http://ifttt.com"" rel=""nofollow"">IFTT...",False,NaN,NaN,NaN,NaN,...,2016-03-23,NaN,"[AGFS, 20,230.00]",agrofresh solutions inc agfs director buys 202...,NaN,NaN,NaN,NaN,NEGATIVE,0.993538
3,Tue Aug 18 20:13:24 +0000 2015,633733455549890560,633733455549890560,RT @AlertTrade: SHORT Seller Alert $AGFS - AGR...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,2015-08-18,{'created_at': 'Tue Aug 18 19:30:15 +0000 2015...,[AGFS],rt short seller alert agfs - agrofresh solutio...,NaN,NaN,NaN,NaN,NEGATIVE,0.996947
4,Wed Feb 10 12:52:57 +0000 2016,697402877053108228,697402877053108228,RT @talent4boards: AgroFresh appoints Marc Las...,"<a href=""https://roundteam.co"" rel=""nofollow"">...",False,NaN,NaN,NaN,NaN,...,2016-02-10,{'created_at': 'Wed Feb 10 12:50:07 +0000 2016...,[AGFS],rt agrofresh appoints marc lasry to as indepen...,NaN,NaN,NaN,NaN,POSITIVE,0.673652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119839,Fri Aug 07 04:46:13 +0000 2015,629513855719215104,629513855719215104,SHAK Shake Shack Inc. 52Wk Low\nhttp://t.co/mX...,"<a href=""http://dlvr.it"" rel=""nofollow"">dlvr.i...",False,NaN,NaN,NaN,NaN,...,2015-08-07,NaN,"[NKE, SHAK, CELG, DWTI]",shak shake shack inc 52wk low shak nke celg dwti,NaN,NaN,NaN,NaN,NEGATIVE,0.998781
119840,Fri Aug 07 13:38:35 +0000 2015,629647830194323456,629647830194323456,HIMX Himax Technologies Inc. Company Info\nhtt...,"<a href=""http://dlvr.it"" rel=""nofollow"">dlvr.i...",False,NaN,NaN,NaN,NaN,...,2015-08-07,NaN,"[KMI, CELG, XOM, HIMX]",himx himax technologies inc company info himx ...,NaN,NaN,NaN,NaN,NEGATIVE,0.988348
119841,Fri Aug 07 07:40:12 +0000 2015,629557640046120960,629557640046120960,KBE SPDR KBW Bank Index ETF E.P.S.\nhttp://t.c...,"<a href=""http://dlvr.it"" rel=""nofollow"">dlvr.i...",False,NaN,NaN,NaN,NaN,...,2015-08-07,NaN,"[FXY, CELG, KBE, VALE]",kbe spdr kbw bank index etf eps kbe vale fxy celg,NaN,NaN,NaN,NaN,NEGATIVE,0.995098
119842,Fri Aug 07 07:50:07 +0000 2015,629560135665344512,629560135665344512,VGHCX Vanguard Specialized Portfolios Health C...,"<a href=""http://dlvr.it"" rel=""nofollow"">dlvr.i...",False,NaN,NaN,NaN,NaN,...,2015-08-07,NaN,"[MNK, CELG, VGHCX, CSCO]",vghcx vanguard specialized portfolios health c...,NaN,NaN,NaN,NaN,NEGATIVE,0.987143


In [ ]:
dataset = all_data.copy()

## Top2Vec

In [ ]:
from top2vec import Top2Vec

In [ ]:
dataset = all_data.copy()

In [ ]:
docs = dataset['text_clean'].to_list()

In [ ]:
len(docs)

119844

In [ ]:
model = Top2Vec(
    documents=docs,
    speed="learn",
    #speed="learn",
    workers=8,
    embedding_model='universal-sentence-encoder',
    topic_merge_delta=0.5
)

2023-12-03 20:17:01,900 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
2023-12-03 20:17:07,028 - top2vec - INFO - Downloading universal-sentence-encoder model
INFO:top2vec:Downloading universal-sentence-encoder model
2023-12-03 20:17:23,087 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2023-12-03 20:17:43,746 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2023-12-03 20:20:44,715 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2023-12-03 20:20:53,222 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


In [ ]:
dataset['topics'] = model.doc_top

In [ ]:
# One hot topics
topics = pd.get_dummies(dataset['topics'], prefix='tp')
cols_topics = list(topics)

In [ ]:
# Embeddings
vectors = model.document_vectors
vectors.shape[1]
embedding = pd.DataFrame(
    model.document_vectors,
    columns=[f't2ve_{str(i+1).zfill(3)}' for i in range(vectors.shape[1])]
)
cols_embedding = list(embedding)

In [ ]:
# Sentiment
dataset = dataset.rename(columns={
    'text_clean sentiment_label':'sentiment_label',
    'text_clean sentiment_score':'sentiment_score'
})

sentiments = pd.get_dummies(all_data['sentiment_label'].str.lower(), prefix='sent')
cols_sentiment = list(sentiments)

In [ ]:
dataset = dataset[[x for x in list(dataset) if x not in cols_topics]]
if cols_topics[0] not in list(dataset):
    dataset = pd.concat([dataset, topics], axis=1)

In [ ]:
dataset = dataset[[x for x in list(dataset) if x not in  cols_embedding]]
if cols_embedding[0] not in list(dataset):
    dataset = pd.concat([dataset, embedding], axis=1)

In [ ]:
dataset = dataset[[x for x in list(dataset) if x not in cols_sentiment]]
if cols_sentiment[0] not in list(dataset):
    dataset = pd.concat([dataset, sentiments], axis=1)

In [ ]:
dataset['created_at'] = pd.to_datetime(dataset['created_at'])
dataset['date'] = dataset['created_at'].dt.date

In [ ]:
dataset['sentiment_score']

0         0.982623
1         0.698450
2         0.993538
3         0.996947
4         0.673652
            ...   
119839    0.998781
119840    0.988348
119841    0.995098
119842    0.987143
119843    0.989086
Name: sentiment_score, Length: 119844, dtype: float64

In [ ]:
to_export = dataset[['created_at','date','ticker','text','text_clean','sentiment_score']+cols_topics+cols_embedding+cols_sentiment]
to_export.to_csv(os.path.join(PATH_DATA, 'tweet_embeddings_all.csv'), index=False)

In [ ]:
to_export.head()

,created_at,date,ticker,text,text_clean,sentiment_score,tp_0,tp_1,tp_2,tp_3,...,t2ve_505,t2ve_506,t2ve_507,t2ve_508,t2ve_509,t2ve_510,t2ve_511,t2ve_512,sent_negative,sent_positive
0,2016-03-01 15:58:13+00:00,2016-03-01,AGFS,clayton: AGFS 4.89 Agrofresh Solutions Inc $AG...,clayton: agfs 489 agrofresh solutions inc agfs...,0.982623,0,0,0,0,...,-0.034576,0.029796,-0.017505,0.007992,-0.040249,-0.008642,-0.046793,-0.021782,1,0
1,2015-11-11 19:14:27+00:00,2015-11-11,AGFS,$AGFS steady accumulation,agfs steady accumulation,0.698450,0,1,0,0,...,0.088483,0.048010,0.006079,-0.018939,-0.021779,0.014671,-0.006702,0.029079,1,0
2,2016-03-23 03:48:58+00:00,2016-03-23,AGFS,AgroFresh Solutions Inc $AGFS Director Buys $2...,agrofresh solutions inc agfs director buys 202...,0.993538,0,0,0,0,...,-0.021881,0.011881,-0.065938,0.021464,0.029966,0.032440,0.019499,-0.022567,1,0
3,2015-08-18 20:13:24+00:00,2015-08-18,AGFS,RT @AlertTrade: SHORT Seller Alert $AGFS - AGR...,rt short seller alert agfs - agrofresh solutio...,0.996947,1,0,0,0,...,0.064251,0.036838,0.048882,0.002030,-0.072927,0.084812,0.020522,-0.023972,1,0
4,2016-02-10 12:52:57+00:00,2016-02-10,AGFS,RT @talent4boards: AgroFresh appoints Marc Las...,rt agrofresh appoints marc lasry to as indepen...,0.673652,0,0,0,0,...,0.068008,0.020154,-0.070788,0.043688,0.059510,0.040473,0.024878,-0.030680,0,1


In [ ]:
to_export = pd.read_csv(os.path.join(PATH_DATA, 'tweet_embeddings_all.csv'))

In [ ]:
keys = ['ticker', 'date']

In [ ]:
to_export[keys]

,ticker,date
0,AGFS,2016-03-01
1,AGFS,2015-11-11
2,AGFS,2016-03-23
3,AGFS,2015-08-18
4,AGFS,2016-02-10
...,...,...
122476,CELG,2015-08-07
122477,CELG,2015-08-07
122478,CELG,2015-08-07
122479,CELG,2015-08-07


In [ ]:
to_export = to_export[to_export['ticker'].astype(str).str.isalpha()]

In [ ]:
keys

['ticker', 'date']

In [ ]:
df_aggregated = pd.concat([
    to_export[keys+cols_topics].groupby(keys).sum(),
    to_export[keys+cols_embedding].groupby(keys).mean(),
    to_export[keys+cols_sentiment].groupby(keys).sum(),
    to_export[keys+['sentiment_score']].groupby(keys).mean()
], axis=1).reset_index()

In [ ]:
df_aggregated.columns  = [x if 'sent' not in x else f'tweet_{x}' for x in df_aggregated]

In [ ]:
df_aggregated

,ticker,date,tp_0,tp_1,tp_2,tp_3,tp_4,tp_5,tp_6,tp_7,...,t2ve_506,t2ve_507,t2ve_508,t2ve_509,t2ve_510,t2ve_511,t2ve_512,tweet_sent_negative,tweet_sent_positive,tweet_sentiment_score
0,AAPL,2013-12-31,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.009038,0.024683,0.014230,-0.005057,0.036838,-0.034250,0.042868,1.0,0.0,0.976285
1,AAPL,2014-01-01,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.006272,0.038048,0.028415,-0.003747,0.028702,-0.046322,0.019313,6.0,2.0,0.945411
2,AAPL,2014-01-02,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003726,0.038268,0.026364,-0.009559,0.031142,-0.054542,-0.013422,18.0,14.0,0.972707
3,AAPL,2014-01-03,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.007831,0.034383,0.044858,-0.010264,0.029918,-0.045999,0.008392,16.0,5.0,0.974000
4,AAPL,2014-01-04,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.014244,0.015553,0.025035,-0.018254,0.044570,-0.054826,0.017356,4.0,1.0,0.969403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29179,XOM,2016-03-25,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,-0.022610,0.034965,0.013589,0.027157,0.022247,0.029645,-0.016359,4.0,0.0,0.993343
29180,XOM,2016-03-28,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005840,0.074975,0.041927,0.028465,0.059962,-0.075009,-0.075033,1.0,0.0,0.995899
29181,XOM,2016-03-29,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.012572,0.020778,0.041295,-0.018412,0.014120,-0.038374,0.004712,9.0,1.0,0.984425
29182,XOM,2016-03-30,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.016277,0.011227,0.036124,-0.010195,0.054641,-0.035273,-0.002741,4.0,0.0,0.990428


In [ ]:
df_aggregated.to_csv(os.path.join(PATH_AGG_DATA, 'tweet_embeddings_all_agg.csv'), index=False)

In [ ]:
"""
df_aggregated = []
for ticker, date in tqdm(combinations):
    df_i = to_export[(to_export['ticker']==ticker) & (to_export['date']==date)]

    topics = []
    embeddings = []
    for i, row in df_i.iterrows():
        topics.append(row['topics'])
        embeddings.append(row['doc_embedding'])

    data_agg = {
        'ticker':ticker,
        'date': date,
        'topics': topics,
        'embedding_t2v': sum(embeddings) / len(embeddings)
    }
    df_aggregated.append(data_agg)
"""

"\ndf_aggregated = []\nfor ticker, date in tqdm(combinations):\n    df_i = to_export[(to_export['ticker']==ticker) & (to_export['date']==date)]\n\n    topics = []\n    embeddings = []\n    for i, row in df_i.iterrows():\n        topics.append(row['topics'])\n        embeddings.append(row['doc_embedding'])\n\n    data_agg = {\n        'ticker':ticker,\n        'date': date,\n        'topics': topics,\n        'embedding_t2v': sum(embeddings) / len(embeddings)\n    }\n    df_aggregated.append(data_agg)\n"

In [ ]:
pd.DataFrame(df_aggregated).to_csv(os.path.join(PATH_DATA, 'tweet_embeddings_all_agg.csv'), index=False)